In [1]:
# Data Hackathon 3.x 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
from keras.datasets import mnist
(X_train, y_train), (X_test, y_test) = mnist.load_data()
X_train = X_train.reshape(X_train.shape[0],784)
X_test = X_test.reshape(X_test.shape[0],784)

Using TensorFlow backend.


In [3]:
#X_train_centered = (X_train-np.mean(X_train, axis=0)/np.std(X_train))
#X_test_centered = (X_test-np.mean(X_test, axis=0)/np.std(X_test))
#del X_train
#del X_test
#print(X_train_centered.shape, y_train.shape)
#print(X_test_centered.shape, y_test.shape)

In [4]:
X_train.shape
X_test.shape

(10000, 784)

In [5]:
y_train.shape

(60000,)

In [6]:
from sklearn.cross_validation import train_test_split
train,x_val,y,y_val = train_test_split(X_train,y_train,
                                       test_size=.3,random_state=123)
train.shape

/root/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


(42000, 784)

In [9]:
import xgboost as xgb

xgb_train = xgb.DMatrix(train,label=y)
xgb_val = xgb.DMatrix(x_val, label=y_val)
xgb_test = xgb.DMatrix(X_test)

In [13]:
params={
    'booster':'gbtree',
    'objective':'multi:softmax',
    'num_class':10,
    'gamma':0.1,
    'max_depth':12,
    'lambda':2,
    'subsample':0.7,
    'colsample_bytree':0.7,
    'min_child_weight':3,
    # 这个参数默认是 1，是每个叶子里面 h 的和至少是多少，对正负样本不均衡时的 0-1 分类而言
    #，假设 h 在 0.01 附近，min_child_weight 为 1 意味着叶子节点中最少需要包含 100 个样本。
    #这个参数非常影响结果，控制叶子节点中二阶导的和的最小值，该参数值越小，越容易 overfitting。
    'silent':0,
    'eta':0.007,
    'seed':1000,
    'nthread':7
}

plst = list(params.items())
num_rounds = 5000
watchlist = [(xgb_train,'train'),(xgb_val,'val')]

model = xgb.train(plst, xgb_train, num_rounds, 
                  watchlist,early_stopping_rounds=100)
model.save_model('./model/xgb.model') # 用于存储训练出的模型
print("best best_ntree_limit",model.best_ntree_limit)

[0]	train-merror:0.0795	val-merror:0.111778
Multiple eval metrics have been passed: 'val-merror' will be used for early stopping.

Will train until val-merror hasn't improved in 100 rounds.
[1]	train-merror:0.060119	val-merror:0.087611
[2]	train-merror:0.051905	val-merror:0.077389
[3]	train-merror:0.048619	val-merror:0.074444
[4]	train-merror:0.046571	val-merror:0.070556
[5]	train-merror:0.045452	val-merror:0.068722
[6]	train-merror:0.043381	val-merror:0.066333
[7]	train-merror:0.043048	val-merror:0.066111
[8]	train-merror:0.042143	val-merror:0.065
[9]	train-merror:0.04169	val-merror:0.0645
[10]	train-merror:0.041952	val-merror:0.064611
[11]	train-merror:0.041476	val-merror:0.063833
[12]	train-merror:0.040119	val-merror:0.0635
[13]	train-merror:0.039524	val-merror:0.062389
[14]	train-merror:0.039524	val-merror:0.061444
[15]	train-merror:0.039286	val-merror:0.061333
[16]	train-merror:0.038833	val-merror:0.060889
[17]	train-merror:0.038619	val-merror:0.060389
[18]	train-merror:0.037762	v

[172]	train-merror:0.019119	val-merror:0.046778
[173]	train-merror:0.019071	val-merror:0.046778
[174]	train-merror:0.018929	val-merror:0.046778
[175]	train-merror:0.018905	val-merror:0.046833
[176]	train-merror:0.01881	val-merror:0.046944
[177]	train-merror:0.018762	val-merror:0.046889
[178]	train-merror:0.018762	val-merror:0.046778
[179]	train-merror:0.01869	val-merror:0.046722
[180]	train-merror:0.018643	val-merror:0.046611
[181]	train-merror:0.0185	val-merror:0.046611
[182]	train-merror:0.0185	val-merror:0.046722
[183]	train-merror:0.018476	val-merror:0.046778
[184]	train-merror:0.018429	val-merror:0.047
[185]	train-merror:0.018381	val-merror:0.046833
[186]	train-merror:0.018357	val-merror:0.046722
[187]	train-merror:0.018381	val-merror:0.046556
[188]	train-merror:0.01831	val-merror:0.046444
[189]	train-merror:0.018286	val-merror:0.0465
[190]	train-merror:0.018143	val-merror:0.046556
[191]	train-merror:0.018119	val-merror:0.046444
[192]	train-merror:0.018095	val-merror:0.046333
[193

[345]	train-merror:0.0115	val-merror:0.041556
[346]	train-merror:0.011429	val-merror:0.041556
[347]	train-merror:0.011405	val-merror:0.041556
[348]	train-merror:0.011381	val-merror:0.041389
[349]	train-merror:0.011286	val-merror:0.041333
[350]	train-merror:0.011238	val-merror:0.041444
[351]	train-merror:0.011238	val-merror:0.041444
[352]	train-merror:0.01119	val-merror:0.0415
[353]	train-merror:0.011167	val-merror:0.041389
[354]	train-merror:0.01119	val-merror:0.041278
[355]	train-merror:0.011167	val-merror:0.041111
[356]	train-merror:0.011143	val-merror:0.041222
[357]	train-merror:0.011119	val-merror:0.041056
[358]	train-merror:0.011095	val-merror:0.041111
[359]	train-merror:0.011071	val-merror:0.041167
[360]	train-merror:0.011048	val-merror:0.041167
[361]	train-merror:0.011048	val-merror:0.041111
[362]	train-merror:0.011	val-merror:0.041056
[363]	train-merror:0.010952	val-merror:0.040944
[364]	train-merror:0.010881	val-merror:0.040944
[365]	train-merror:0.01081	val-merror:0.040722
[3

[518]	train-merror:0.006738	val-merror:0.037167
[519]	train-merror:0.006714	val-merror:0.037167
[520]	train-merror:0.00669	val-merror:0.037222
[521]	train-merror:0.00669	val-merror:0.037222
[522]	train-merror:0.006667	val-merror:0.037222
[523]	train-merror:0.006643	val-merror:0.037056
[524]	train-merror:0.006643	val-merror:0.037056
[525]	train-merror:0.006643	val-merror:0.037056
[526]	train-merror:0.006595	val-merror:0.036889
[527]	train-merror:0.006571	val-merror:0.036889
[528]	train-merror:0.006548	val-merror:0.036889
[529]	train-merror:0.0065	val-merror:0.036889
[530]	train-merror:0.006524	val-merror:0.036833
[531]	train-merror:0.006452	val-merror:0.036889
[532]	train-merror:0.006452	val-merror:0.036889
[533]	train-merror:0.006452	val-merror:0.036889
[534]	train-merror:0.006452	val-merror:0.036889
[535]	train-merror:0.006429	val-merror:0.036833
[536]	train-merror:0.006381	val-merror:0.036833
[537]	train-merror:0.006357	val-merror:0.036833
[538]	train-merror:0.006357	val-merror:0.036

[691]	train-merror:0.003405	val-merror:0.034111
[692]	train-merror:0.003381	val-merror:0.034056
[693]	train-merror:0.003333	val-merror:0.034167
[694]	train-merror:0.003333	val-merror:0.034111
[695]	train-merror:0.003333	val-merror:0.034056
[696]	train-merror:0.003333	val-merror:0.033944
[697]	train-merror:0.003333	val-merror:0.033833
[698]	train-merror:0.003333	val-merror:0.033944
[699]	train-merror:0.003333	val-merror:0.033889
[700]	train-merror:0.003286	val-merror:0.033778
[701]	train-merror:0.003286	val-merror:0.033722
[702]	train-merror:0.003286	val-merror:0.033667
[703]	train-merror:0.003262	val-merror:0.033667
[704]	train-merror:0.003262	val-merror:0.033667
[705]	train-merror:0.003238	val-merror:0.033667
[706]	train-merror:0.003262	val-merror:0.033667
[707]	train-merror:0.003238	val-merror:0.033667
[708]	train-merror:0.003238	val-merror:0.033556
[709]	train-merror:0.003238	val-merror:0.033611
[710]	train-merror:0.003214	val-merror:0.0335
[711]	train-merror:0.003214	val-merror:0.0

...

[1546]	train-merror:9.5e-05	val-merror:0.026111
[1547]	train-merror:9.5e-05	val-merror:0.026111
[1548]	train-merror:9.5e-05	val-merror:0.026111
[1549]	train-merror:9.5e-05	val-merror:0.026111
[1550]	train-merror:9.5e-05	val-merror:0.026111
[1551]	train-merror:9.5e-05	val-merror:0.026111
[1552]	train-merror:9.5e-05	val-merror:0.026056
[1553]	train-merror:9.5e-05	val-merror:0.026056
[1554]	train-merror:9.5e-05	val-merror:0.026056
[1555]	train-merror:9.5e-05	val-merror:0.026111
[1556]	train-merror:9.5e-05	val-merror:0.026111
[1557]	train-merror:9.5e-05	val-merror:0.026111
[1558]	train-merror:9.5e-05	val-merror:0.026
[1559]	train-merror:7.1e-05	val-merror:0.026111
[1560]	train-merror:7.1e-05	val-merror:0.026056
[1561]	train-merror:7.1e-05	val-merror:0.026056
[1562]	train-merror:7.1e-05	val-merror:0.026056
[1563]	train-merror:7.1e-05	val-merror:0.026
[1564]	train-merror:7.1e-05	val-merror:0.026
[1565]	train-merror:7.1e-05	val-merror:0.026056
[1566]	train-merror:7.1e-05	val-merror:0.026056
[

[1730]	train-merror:0	val-merror:0.025944
[1731]	train-merror:0	val-merror:0.025944
[1732]	train-merror:0	val-merror:0.025889
[1733]	train-merror:0	val-merror:0.025833
[1734]	train-merror:0	val-merror:0.025833
[1735]	train-merror:0	val-merror:0.025833
[1736]	train-merror:0	val-merror:0.025889
[1737]	train-merror:0	val-merror:0.025889
[1738]	train-merror:0	val-merror:0.025889
[1739]	train-merror:0	val-merror:0.025833
[1740]	train-merror:0	val-merror:0.025833
[1741]	train-merror:0	val-merror:0.025833
[1742]	train-merror:0	val-merror:0.025833
[1743]	train-merror:0	val-merror:0.025833
[1744]	train-merror:0	val-merror:0.025833
[1745]	train-merror:0	val-merror:0.025833
[1746]	train-merror:0	val-merror:0.025833
[1747]	train-merror:0	val-merror:0.025833
[1748]	train-merror:0	val-merror:0.025833
[1749]	train-merror:0	val-merror:0.025833
[1750]	train-merror:0	val-merror:0.025833
[1751]	train-merror:0	val-merror:0.025833
[1752]	train-merror:0	val-merror:0.025833
[1753]	train-merror:0	val-merror:0

[1927]	train-merror:0	val-merror:0.025222
[1928]	train-merror:0	val-merror:0.025222
[1929]	train-merror:0	val-merror:0.025222
[1930]	train-merror:0	val-merror:0.025222
[1931]	train-merror:0	val-merror:0.025222
[1932]	train-merror:0	val-merror:0.025222
[1933]	train-merror:0	val-merror:0.025167
[1934]	train-merror:0	val-merror:0.025167
[1935]	train-merror:0	val-merror:0.025111
[1936]	train-merror:0	val-merror:0.025111
[1937]	train-merror:0	val-merror:0.025111
[1938]	train-merror:0	val-merror:0.025222
[1939]	train-merror:0	val-merror:0.025167
[1940]	train-merror:0	val-merror:0.025111
[1941]	train-merror:0	val-merror:0.025222
[1942]	train-merror:0	val-merror:0.025278
[1943]	train-merror:0	val-merror:0.025278
[1944]	train-merror:0	val-merror:0.025167
[1945]	train-merror:0	val-merror:0.025278
[1946]	train-merror:0	val-merror:0.025278
[1947]	train-merror:0	val-merror:0.025333
[1948]	train-merror:0	val-merror:0.025278
[1949]	train-merror:0	val-merror:0.025222
[1950]	train-merror:0	val-merror:0

[2123]	train-merror:0	val-merror:0.024944
[2124]	train-merror:0	val-merror:0.024944
[2125]	train-merror:0	val-merror:0.024944
[2126]	train-merror:0	val-merror:0.024944
[2127]	train-merror:0	val-merror:0.024889
[2128]	train-merror:0	val-merror:0.024889
[2129]	train-merror:0	val-merror:0.024889
[2130]	train-merror:0	val-merror:0.024889
[2131]	train-merror:0	val-merror:0.024944
[2132]	train-merror:0	val-merror:0.024944
[2133]	train-merror:0	val-merror:0.024944
[2134]	train-merror:0	val-merror:0.024944
[2135]	train-merror:0	val-merror:0.024944
[2136]	train-merror:0	val-merror:0.024944
[2137]	train-merror:0	val-merror:0.024889
[2138]	train-merror:0	val-merror:0.024889
[2139]	train-merror:0	val-merror:0.024944
[2140]	train-merror:0	val-merror:0.024889
[2141]	train-merror:0	val-merror:0.024889
[2142]	train-merror:0	val-merror:0.024889
[2143]	train-merror:0	val-merror:0.024889
[2144]	train-merror:0	val-merror:0.024889
[2145]	train-merror:0	val-merror:0.024833
[2146]	train-merror:0	val-merror:0

XGBoostError: b'[07:04:59] src/io/local_filesys.cc:149: Check failed: allow_null  LocalFileSystem: fail to open "./model/xgb.model"'

In [ ]:
preds = model.predict(xgb_test,ntree_limit=model.best_ntree_limit)

np.savetxt('xgb_submission.csv',np.c_[range(1,len(tests)+1),preds],delimiter=',',header='ImageId,Label',comments='',fmt='%d')

#输出运行时长
cost_time = time.time()-start_time
print("xgboost success!",'\n',"cost time:",cost_time,"(s)......")